In [9]:
import sqlite3
import pandas as pd
import sqlite3
import pandas as pd
from IPython.display import display

DB_PATH = 'escola_ingles.db'

def executar_query(sql_query):
    """Função Helper para conectar e retornar DataFrame."""
    try:
        conn = sqlite3.connect(DB_PATH)
        df_resultado = pd.read_sql_query(sql_query, conn)
        conn.close()
        print(f"-> Query executada! Retornou {len(df_resultado)} linhas.")
        return df_resultado
    except Exception as e:
        print(f"❌ Erro na Query SQL:\n{e}")

def resetar_banco():
    """Recria o banco de dados do zero com dados da Escola de Inglês."""

    print("🚀 Iniciando o reset do banco de dados...")
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    print(f"🔄 Resetando o Banco de Dados '{DB_PATH}'...")
    cursor.execute("DROP TABLE IF EXISTS funcionarios")
    cursor.execute("""
        CREATE TABLE funcionarios (
            id_func INTEGER PRIMARY KEY,
            nome TEXT,
            cargo TEXT,
            idioma_principal TEXT,
            valor_hora_aula REAL,
            data_contratacao TEXT
        )
    """)
    
    staff_data = [
        (1, 'Mr. John Smith', 'Professor', 'Inglês', 85.50, '2019-02-10'),
        (2, 'Sarah Connor', 'Coordenadora', 'Inglês', 120.00, '2018-05-15'),
        (3, 'Juan Garcia', 'Professor', 'Espanhol', 80.00, '2021-03-20'),
        (4, 'Amanda Lee', 'Recepcionista', 'Português', 25.00, '2023-01-10'),
        (5, 'Mike Ross', 'Professor', 'Inglês', 90.00, '2022-08-05'),
        (6, 'Pierre Dupont', 'Professor', 'Francês', 95.00, '2020-11-01')
    ]
    cursor.executemany("INSERT INTO funcionarios VALUES (?,?,?,?,?,?)", staff_data)
    
    cursor.execute("DROP TABLE IF EXISTS cursos")
    cursor.execute("""
        CREATE TABLE cursos (
            id_curso INTEGER PRIMARY KEY,
            nome_curso TEXT,
            nivel TEXT,
            dias_semana TEXT,
            capacidade_maxima INTEGER,
            mensalidade REAL
        )
    """)
    
    cursos_data = [
        (101, 'General English', 'Básico (A1)', 'Seg/Qua', 20, 350.00),
        (102, 'General English', 'Intermediário (B1)', 'Ter/Qui', 15, 380.00),
        (103, 'Business English', 'Avançado (C1)', 'Sáb', 10, 550.00),
        (104, 'Kids Fun', 'Iniciante', 'Seg/Qua', 12, 320.00),
        (105, 'Conversation Club', 'Avançado (C2)', 'Sex', 8, 200.00),
        (106, 'Intensive IELTS', 'Preparatório', 'Ter/Qui', 10, 600.00)
    ]
    cursor.executemany("INSERT INTO cursos VALUES (?,?,?,?,?,?)", cursos_data)

    conn.commit()
    conn.close()
    print(f"✅ Banco de Dados '{DB_PATH}' da Global Speak Academy criado!")



resetar_banco()


🚀 Iniciando o reset do banco de dados...
🔄 Resetando o Banco de Dados 'escola_ingles.db'...
✅ Banco de Dados 'escola_ingles.db' da Global Speak Academy criado!


In [10]:
DB_PATH = 'escola_v2.db'

def resetar_banco_relacional():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    # --- TABELA 1: ALUNOS (Entidade Forte) ---
    # PK: id_aluno
    cursor.execute("DROP TABLE IF EXISTS alunos")
    cursor.execute("""
        CREATE TABLE alunos (
            id_aluno INTEGER PRIMARY KEY,  -- PK: Identificador Único
            nome TEXT,
            email TEXT,
            cidade TEXT,
            data_nascimento TEXT
        )
    """)
    
    # --- TABELA 2: CURSOS (Entidade Forte) ---
    # PK: id_curso
    cursor.execute("DROP TABLE IF EXISTS cursos")
    cursor.execute("""
        CREATE TABLE cursos (
            id_curso INTEGER PRIMARY KEY, -- PK
            nome_curso TEXT,
            nivel TEXT,
            preco_mensal REAL
        )
    """)
    
    # --- TABELA 3: MATRICULAS (Tabela Associativa/Fato) ---
    # Aqui mora o conceito de Chave Estrangeira (FK)
    # Ela liga um Aluno (id_aluno) a um Curso (id_curso)
    cursor.execute("DROP TABLE IF EXISTS matriculas")
    cursor.execute("""
        CREATE TABLE matriculas (
            id_matricula INTEGER PRIMARY KEY,
            id_aluno INTEGER, -- FK (Aponta para tabela Alunos)
            id_curso INTEGER, -- FK (Aponta para tabela Cursos)
            data_matricula TEXT,
            status TEXT,
            nota_final REAL,
            FOREIGN KEY (id_aluno) REFERENCES alunos (id_aluno),
            FOREIGN KEY (id_curso) REFERENCES cursos (id_curso)
        )
    """)
    
    # Populando dados (Mock Data)
    # Alunos
    alunos = [
        (1, 'Ana Silva', 'ana@email.com', 'São Paulo', '1995-03-10'),
        (2, 'Bruno Souza', 'bruno@email.com', 'Rio de Janeiro', '2000-01-20'),
        (3, 'Carla Diaz', 'carla@email.com', 'São Paulo', '1998-07-15'),
        (4, 'Daniel Rocha', 'daniel@email.com', 'Belo Horizonte', '1990-11-30'),
        (5, 'Eduardo Mello', 'edu@email.com', 'Curitiba', '2002-05-10')
    ]
    cursor.executemany("INSERT INTO alunos VALUES (?,?,?,?,?)", alunos)
    
    # Cursos
    cursos = [
        (101, 'Inglês', 'Básico', 300.00),
        (102, 'Inglês', 'Avançado', 450.00),
        (103, 'Espanhol', 'Intermediário', 350.00),
        (104, 'Francês', 'Básico', 320.00)
    ]
    cursor.executemany("INSERT INTO cursos VALUES (?,?,?,?)", cursos)
    
    # Matriculas (Note que usamos números, não nomes! Isso é a FK em ação)
    # Ex: Aluno 1 (Ana) faz o curso 101 (Inglês Básico)
    matriculas = [
        (1001, 1, 101, '2023-01-15', 'Ativo', 8.5),
        (1002, 1, 104, '2023-02-20', 'Ativo', 9.0), # Ana faz Francês tb
        (1003, 2, 102, '2023-01-10', 'Cancelado', 4.5),
        (1004, 3, 101, '2023-03-01', 'Ativo', 7.8),
        (1005, 4, 103, '2023-01-20', 'Concluido', 10.0),
        (1006, 5, 101, '2023-02-15', 'Ativo', 6.5)
    ]
    cursor.executemany("INSERT INTO matriculas VALUES (?,?,?,?,?,?)", matriculas)

    conn.commit()
    conn.close()
    print("✅ Banco Relacional Criado com Sucesso!")

def executar_query(sql):
    conn = sqlite3.connect(DB_PATH)
    try:
        df = pd.read_sql_query(sql, conn)
        return df
    finally:
        conn.close()
resetar_banco_relacional()

✅ Banco Relacional Criado com Sucesso!
